# Data Loading

In [ ]:
## Data_preprocessing_Economic_indicator

from google.colab import drive
drive.mount( '/content/drive' )

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Full_df = pd.read_csv("/content/drive/MyDrive/공모전/B.I. 공모전/B.I Contest/데이터/최종 데이터/최종/dataset.csv")

Selected_df = pd.read_csv("/content/drive/MyDrive/공모전/B.I. 공모전/B.I Contest/데이터/최종 데이터/최종/data_selected.csv")

Full_df.rename(columns = {'Unnamed: 0' : '날짜'}, inplace = True)
Full_df['날짜'] = pd.to_datetime(Full_df['날짜'])

Selected_df.rename(columns = {'Unnamed: 0' : '날짜'}, inplace = True)
Selected_df['날짜'] = pd.to_datetime(Selected_df['날짜'])


print(Full_df)

# Data Preprocessing

In [ ]:
# change Float 64 to Float 32

def Transform_to_float32( data, num ) :

    # num : 바꿀 x 변수 개수, 태양광발전량부터 현재수요+태양광은 바꾸면 안됨

    for i in range( num ) :
        col = data.columns[ i + 1 ]
        data[ col ] = data[ col ].astype( np.float32 )

    return data

float_32_full_df = Transform_to_float32(Full_df, 12)
float_32_selected_df = Transform_to_float32(Selected_df, 9)

print(float_32_full_df.info())
print(float_32_selected_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077408 entries, 0 to 1077407
Data columns (total 16 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   날짜            1077408 non-null  datetime64[ns]
 1   기온            1077408 non-null  float32       
 2   강수량           1077408 non-null  float32       
 3   습도            1077408 non-null  float32       
 4   일조            1077408 non-null  float32       
 5   일사            1077408 non-null  float32       
 6   적설            1077408 non-null  float32       
 7   환율            1077408 non-null  float32       
 8   GDP           1077408 non-null  float32       
 9   공휴일           1077408 non-null  float32       
 10  주말            1077408 non-null  float32       
 11  평일            1077408 non-null  float32       
 12  electric      1077408 non-null  float32       
 13  태양광 발전량(MWh)  1077408 non-null  float64       
 14  현재수요(MW)      1077408 non-null  float64       
 15

In [ ]:
def seed_everything(seed: int = 42):

    import os
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore


In [ ]:
# HyperParameter Tuning list, define device

CFG = {'ip' : 2016,
       'lr' : 0.001,
       'epoch' : 30,
        'seed' : 100}


# 돌려보기

In [ ]:
# Data split

# input_df : input_df
# train_start : 학습 데이터 시작 yyyy-mm-dd hh:mm:ss
# val_start : 검증 데이터 시작 yyyy-mm-dd hh:mm:ss
# test_start : 테스트 데이터 시작 yyyy-mm-dd hh:mm:ss

def split_train_val_test(input_df, train_start, val_start, test_start, test_last, ip) -> pd.DataFrame:

    import copy
    from datetime import datetime, timedelta

    copy_df = copy.deepcopy(input_df)
    copy_df = copy_df.set_index('날짜')

    date_format = '%Y-%m-%d %H:%M:%S'

    train_date = datetime.strptime(train_start, date_format)
    val_date = datetime.strptime(val_start, date_format)
    test_date = datetime.strptime(test_start, date_format)

    train = copy_df[train_date : val_date]
    val = copy_df[val_date + timedelta(minutes = 5) : test_date - timedelta(minutes = 5 * ip)]
    test = copy_df[test_date - timedelta(minutes = 5 * ip) + timedelta(minutes = 5) : test_last]

    return train, val, test

X_train, X_val, X_test = split_train_val_test(Selected_df, '2018-01-01 00:00:00', '2022-03-12 23:55:00', '2023-03-12 23:55:00', '2023-03-19 23:55:00', ip = CFG['ip'])

Y_test = X_test['현재수요(MW)']['2023-03-13 00:00:00' : '2023-03-19 23:55:00']
Y_test_reset_index = Y_test.reset_index(drop = True)
Y_test_reset_index = Y_test_reset_index.to_numpy()

Sunpower_test= X_test['태양광 발전량(MWh)']['2023-03-13 00:00:00' : '2023-03-19 23:55:00']
Sunpower_test_reset_index = Sunpower_test.reset_index(drop = True)

print(Y_test)
print(Sunpower_test)

날짜
2023-03-13 00:00:00    58782.03
2023-03-13 00:05:00    58954.44
2023-03-13 00:10:00    58710.58
2023-03-13 00:15:00    58298.44
2023-03-13 00:20:00    58263.70
                         ...   
2023-03-19 23:35:00    58627.16
2023-03-19 23:40:00    57941.24
2023-03-19 23:45:00    57726.21
2023-03-19 23:50:00    57755.40
2023-03-19 23:55:00    57323.79
Name: 현재수요(MW), Length: 2016, dtype: float64
날짜
2023-03-13 00:00:00    0.0001
2023-03-13 00:05:00    0.0001
2023-03-13 00:10:00    0.0001
2023-03-13 00:15:00    0.0001
2023-03-13 00:20:00    0.0001
                        ...  
2023-03-19 23:35:00    0.1905
2023-03-19 23:40:00    0.1905
2023-03-19 23:45:00    0.1905
2023-03-19 23:50:00    0.1905
2023-03-19 23:55:00    0.1905
Name: 태양광 발전량(MWh), Length: 2016, dtype: float64


In [ ]:
# data_scaling

def data_scaling(X_train : pd.DataFrame, X_val : pd.DataFrame, X_test : pd.DataFrame, method) -> pd.DataFrame :


# X_train : train_data_set
# X_val : Val_data_set
# X_test : test_data_set
# method : min-max, standard

# output : X_train_scaled, X_val_scaled, X_test_scaled

    drop_list = ['공휴일', '주말', '적설']

    temp_X_train = X_train[drop_list]
    temp_X_val = X_val[drop_list]
    temp_X_test = X_test[drop_list]

    X_train = X_train.drop(columns = drop_list, axis = 1)
    X_val = X_val.drop(columns = drop_list, axis = 1)
    X_test = X_test.drop(columns = drop_list, axis = 1)

    if method == 'standard':

        from sklearn.preprocessing import StandardScaler

        std = StandardScaler()
        std.fit(X_train)

        X_train_scaled = std.transform(X_train)
        X_val_scaled = std.transform(X_val)
        X_test_scaled = std.transform(X_test)

        X_train_scaled_df = pd.DataFrame(index = X_train.index, data = X_train_scaled, columns = X_train.columns)
        X_train_scaled_df = pd.concat([X_train_scaled_df, temp_X_train], axis = 1)

        X_val_scaled_df = pd.DataFrame(index = X_val.index, data = X_val_scaled, columns = X_train.columns)
        X_val_scaled_df = pd.concat([X_val_scaled_df, temp_X_val], axis = 1)

        X_test_scaled_df = pd.DataFrame(index = X_test.index, data = X_test_scaled, columns = X_train.columns)
        X_test_scaled_df = pd.concat([X_test_scaled_df, temp_X_test], axis = 1)

        return X_train_scaled_df, X_val_scaled_df, X_test_scaled_df, std

    elif method == 'min-max':

        from sklearn.preprocessing import MinMaxScaler

        min_max = MinMaxScaler()
        min_max.fit(X_train)

        X_train_scaled = min_max.transform(X_train)
        X_val_scaled = min_max.transform(X_val)
        X_test_scaled = min_max.transform(X_test)

        X_train_scaled_df = pd.DataFrame(index = X_train.index, data = X_train_scaled, columns = X_train.columns)
        X_train_scaled_df = pd.concat([X_train_scaled_df, temp_X_train], axis = 1)

        X_val_scaled_df = pd.DataFrame(index = X_val.index, data = X_val_scaled, columns = X_train.columns)
        X_val_scaled_df = pd.concat([X_val_scaled_df, temp_X_val], axis = 1)

        X_test_scaled_df = pd.DataFrame(index = X_test.index, data = X_test_scaled, columns = X_train.columns)
        X_test_scaled_df = pd.concat([X_test_scaled_df, temp_X_test], axis = 1)

        return X_train_scaled_df, X_val_scaled_df, X_test_scaled_df, min_max


X_train_scaled, X_val_scaled, X_test_scaled, scalar = data_scaling(X_train, X_val, X_test, method = 'min-max')

In [ ]:
X_train_scaled.describe()

,기온,강수량,습도,PC_일조일사,환율,PC_GDP전기차,태양광 발전량(MWh),현재수요(MW),현재수요+태양광,공휴일,주말,적설
count,441216.000000,441216.000000,441216.000000,441216.000000,441216.000000,441216.000000,441216.000000,441216.000000,441216.000000,441216.000000,441216.000000,441216.000000
mean,0.552836,0.010459,0.626833,0.245593,0.435661,0.391234,0.128436,0.452852,0.452825,0.042428,0.272846,0.003546
std,0.194991,0.043060,0.225059,0.328560,0.205097,0.216081,0.163727,0.166472,0.166486,0.201564,0.445423,0.017299
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.394711,0.000000,0.461512,0.000000,0.288852,0.226500,0.001278,0.327612,0.327577,0.000000,0.000000,0.000000
50%,0.559999,0.000000,0.650382,0.002800,0.426044,0.318831,0.055119,0.439307,0.439274,0.000000,0.000000,0.000000
75%,0.719206,0.000262,0.818756,0.531791,0.606729,0.537399,0.209070,0.558728,0.558711,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.294400


In [ ]:
## 필요한 라이브러리 설치

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
seed_everything(CFG['seed'])

In [ ]:
# CustomDataset

class CustomDataSet(Dataset):
    def __init__(self, DataSet, ip, op, stride):

        self.DataSet = DataSet
        self.ip = ip
        self.op = op
        self.stride = stride
        self.data_list = []
        self.label_list = []

        print('Data Pre-processing..')

        L = len(DataSet)
        num_samples = (L - ip - op) // stride + 1

        drop_list = ['태양광 발전량(MWh)', '현재수요(MW)']


        self.input_df = DataSet.drop(columns = drop_list, axis = 1)
        self.target_df = DataSet['현재수요+태양광']

        for idx in tqdm(range(num_samples)):

            Start_x = stride * idx
            End_x = Start_x + ip
            temp_time_data = self.input_df[Start_x : End_x]

            self.data_list.append(torch.Tensor(temp_time_data.values))

            Start_y = stride * idx + ip
            End_y = Start_y + op
            temp_label_data = self.target_df[Start_y : End_y]
            self.label_list.append(torch.Tensor(temp_label_data.values))

        print('\n')
        print("DONE!!")

    def __getitem__(self, index):
        data = self.data_list[index]
        label = self.label_list[index]

        return data, label

    def __len__(self):
        return len(self.data_list)


In [ ]:
# CustomDataSet_DAE : DAE 활용

class CustomDataSet_DAE(Dataset):
    def __init__(self, DataSet, ip, op, stride):

        self.DataSet = DataSet
        self.ip = ip
        self.op = op
        self.stride = stride
        self.data_list = []
        self.label_list = []

        print('Data Pre-processing..')

        L = len(DataSet)
        num_samples = (L - ip - op) // stride + 1

        drop_list = ['태양광 발전량(MWh)', '현재수요(MW)', '현재수요+태양광', '적설']


        self.input_df = DataSet.drop(columns = drop_list, axis = 1)
        self.target_df = DataSet['DAE']

        for idx in tqdm(range(num_samples)):

            Start_x = stride * idx
            End_x = Start_x + ip
            temp_time_data = self.input_df[Start_x : End_x]

            self.data_list.append(torch.Tensor(temp_time_data.values))

            Start_y = stride * idx + ip
            End_y = Start_y + op
            temp_label_data = self.target_df[Start_y : End_y]
            self.label_list.append(torch.Tensor(temp_label_data.values))

        print('\n')
        print("DONE!!")

    def __getitem__(self, index):
        data = self.data_list[index]
        label = self.label_list[index]

        return data, label

    def __len__(self):
        return len(self.data_list)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class GRU_model(nn.Module):
    def __init__(self):
        super(GRU_model, self).__init__()
        self.gru = nn.GRU(input_size = 10, hidden_size = 256, num_layers = 6, batch_first=True, bidirectional=False)
        self.classifier = nn.Sequential(
            nn.Linear(256, 72)
        )

    def forward(self, x):
        hidden, _ = self.gru(x)
        output = self.classifier(hidden[:,-1,:])
        return output


def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)

    # LOSS MAPE로 변경 필요

    criterion = nn.L1Loss().to(device)
    # autograd.set_detect_anomaly(True)

    best_loss = 9999
    best_model = None
    for epoch in range(1, CFG['epoch']):
        model.train()
        train_loss = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            #Y = Y.unsqueeze(1)
            optimizer.zero_grad()

            output = model(X)
            #output = output.unsqueeze(1)

            loss = criterion(output, Y)
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation(model, val_loader, criterion, device)

        print('epoch : ', epoch)
        print(f'Train Loss : [{np.mean(train_loss):.5f}] Valid Loss : [{val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step()

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
    return best_model

In [ ]:
# validation

def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.float().to(device)
            Y = Y.float().to(device)
            model_pred = model(X)
            loss = criterion(model_pred, Y)

            val_loss.append(loss.item())

    return np.mean(val_loss)

In [ ]:
# Inverse_scalar_transformation

def Inverse_scalar_transformation(input_df, scalar, scalar_type) -> pd.Series:

    # input_df : input_data(np.array)
    # scalar_type : min-max, standard

    real_pred_result = []

    if scalar_type == 'min-max':

        min = scalar.data_min_[-1]
        max = scalar.data_max_[-1]

        for i in input_df:

            temp_value = min + i * (max-min)

            real_pred_result.append(temp_value)

        real_pred_result = pd.Series(real_pred_result)


    if scalar_type == 'standard':

        mean = scalar.mean_[-1]
        std = np.sqrt(scalar.var_[-1])

        for i in input_df:

            temp_value = mean + i * (std)
            real_pred_result.append(temp_value)

        real_pred_result = pd.Series(real_pred_result)

    return real_pred_result

In [ ]:
def MAPE(y_test, y_pred):

    # y_test : real
    # y_pred : prediction

	return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [ ]:
# Test_result 구하기

def Get_test_result(model, test_loader, device):

    # MAPE LOSS
    pred_result = []

    model = model

    for X, Y in tqdm(iter(test_loader)):
        X = X.float().to(device)
        Y = Y.float().to(device)

        model_pred = model(X)

        for i in model_pred.flatten():

            pred_result.append(i.item())

    return pred_result

In [ ]:
train_dataset = CustomDataSet(X_train_scaled, ip = CFG['ip'], op = 72, stride = 72)
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle=False)

val_dataset = CustomDataSet(X_val_scaled, ip = CFG['ip'], op = 72, stride = 72)
val_loader = DataLoader(val_dataset, batch_size = 64, shuffle = False)

test_dataset = CustomDataSet(X_test_scaled, ip = CFG['ip'], op = 72, stride = 72)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

Data Pre-processing..


100%|██████████| 6100/6100 [00:02<00:00, 2887.83it/s]




DONE!!
Data Pre-processing..


100%|██████████| 1404/1404 [00:00<00:00, 3077.13it/s]




DONE!!
Data Pre-processing..


100%|██████████| 28/28 [00:00<00:00, 2421.60it/s]



DONE!!


In [ ]:
model = GRU_model()
model.eval()
print(model)
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG['lr'])
scheduler = None
best_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

In [ ]:
test_result = Get_test_result(best_model, test_loader, device)
print("\n")
print("Test Result Done")
print(test_result)

In [ ]:
real_test_result = Inverse_scalar_transformation(test_result, scalar, 'min-max')

real_test_result = real_test_result - Sunpower_test_reset_index
real_test_result = real_test_result.to_numpy()
print("\n")
print("Real Test Result Done")
print(type(real_test_result))

In [ ]:
mape = MAPE(Y_test_reset_index, real_test_result)
print("\n")
print("MAPE Done")
print(mape)